# Options Position Monitor & Trading Cheat Sheet

Real-time monitoring of options positions with technical analysis, Greeks, and key price levels.

**Refresh this notebook daily to update prices and analysis**

---


In [28]:
# Cell 1: Setup and Imports
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import re
from datetime import datetime, date, timedelta
import warnings
from scipy.stats import norm
import os

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

# Color scheme for charts
COLORS = {
    "profit": "#00ff00",
    "loss": "#ff0000",
    "neutral": "#808080",
    "breakeven": "#ffa500",
    "target": "#4169e1",
    "entry": "#ffff00",
    "ma_20": "#4169e1",
    "ma_50": "#ff6347",
}

## Configuration Parameters

**Adjust these values to customize your monitoring:**


In [29]:
# Cell 2: Configuration
CSV_FILES = [
    "Individual-Positions-2026-02-11-223113.csv",
    "Roth Contributory IRA-Positions-2026-02-11-223123.csv",
]

# Risk-free rate (approximate 10-year treasury)
RISK_FREE_RATE = 0.045

# Technical analysis parameters
VOLATILITY_PERIOD = 30  # days for volatility calculation
ATR_PERIOD = 14  # days for Average True Range

# Profit target percentages
PROFIT_TARGETS = [0.25, 0.50, 1.00, 2.00]  # 25%, 50%, 100%, 200%

print(f"📊 Options Monitor Initialized")
print(f"📁 Processing {len(CSV_FILES)} position files")
print(f"📅 Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📊 Options Monitor Initialized
📁 Processing 2 position files
📅 Last Updated: 2026-02-11 23:03:17


## Load and Parse Positions

Extract options positions from CSV files and calculate key metrics.


In [30]:
# Cell 3: Parse CSV Files
def parse_options_csv(file_path):
    """Parse options positions from broker CSV export"""
    try:
        df = pd.read_csv(file_path, skiprows=1)
        options_mask = df["Security Type"] == "Option"
        options_df = df[options_mask].copy()

        if options_df.empty:
            return pd.DataFrame()

        parsed_data = []
        for _, row in options_df.iterrows():
            symbol = str(row["Symbol"])
            pattern = r"([A-Z]+)\s+(\d{2}/\d{2}/\d{4})\s+([\d.]+)\s+([CP])"
            match = re.match(pattern, symbol)

            if match:
                ticker = match.group(1)
                exp_date = datetime.strptime(match.group(2), "%m/%d/%Y").date()
                strike = float(match.group(3))
                opt_type = match.group(4)

                # Clean price values (remove $ and ,)
                price = float(str(row["Price"]).replace("$", "").replace(",", ""))
                qty = int(row["Qty (Quantity)"])
                cost_basis = float(
                    str(row["Cost Basis"]).replace("$", "").replace(",", "")
                )
                market_value = float(
                    str(row["Mkt Val (Market Value)"]).replace("$", "").replace(",", "")
                )
                gain = float(
                    str(row["Gain $ (Gain/Loss $)"]).replace("$", "").replace(",", "")
                )
                gain_pct = float(str(row["Gain % (Gain/Loss %)"]).replace("%", ""))
                dte = int(row["DTE"]) if pd.notna(row["DTE"]) else None

                parsed_data.append(
                    {
                        "ticker": ticker,
                        "symbol": symbol,
                        "option_type": "CALL" if opt_type == "C" else "PUT",
                        "strike": strike,
                        "expiration": exp_date,
                        "dte": dte,
                        "quantity": qty,
                        "price_per_contract": price,
                        "total_cost": cost_basis,
                        "market_value": market_value,
                        "unrealized_pl": gain,
                        "unrealized_pl_pct": gain_pct,
                    }
                )

        return pd.DataFrame(parsed_data)
    except Exception as e:
        print(f"❌ Error parsing {file_path}: {e}")
        return pd.DataFrame()


# Load all positions
all_positions = []
for csv_file in CSV_FILES:
    print(f"📂 Loading {csv_file}...")
    df = parse_options_csv(csv_file)
    if not df.empty:
        all_positions.append(df)
        print(f"   ✓ Found {len(df)} options position(s)")
    else:
        print(f"   ⚠ No options found in {csv_file}")

if not all_positions:
    print("\n❌ No options positions loaded. Check CSV files.")
    positions = pd.DataFrame()
else:
    positions = pd.concat(all_positions, ignore_index=True)
    print(f"\n✅ Total Positions Loaded: {len(positions)}\n")
    display(
        positions[
            ["ticker", "option_type", "strike", "quantity", "price_per_contract", "dte"]
        ].head()
    )

📂 Loading Individual-Positions-2026-02-11-223113.csv...
   ✓ Found 1 options position(s)
📂 Loading Roth Contributory IRA-Positions-2026-02-11-223123.csv...
   ✓ Found 1 options position(s)

✅ Total Positions Loaded: 2



,ticker,option_type,strike,quantity,price_per_contract,dte
0,MSFT,CALL,260.0,1,151.300,219
1,MSFT,CALL,290.0,1,124.175,219


## Calculate Breakeven Prices

Breakeven = Strike Price + Premium Paid (for calls)


In [31]:
# Cell 4: Breakeven Calculations
def calculate_breakeven(strike, price_per_contract, option_type):
    """Calculate breakeven price for an option"""
    if option_type == "CALL":
        return strike + price_per_contract
    else:
        return strike - price_per_contract


# Add breakeven column
positions["breakeven_price"] = positions.apply(
    lambda row: calculate_breakeven(
        row["strike"], row["price_per_contract"], row["option_type"]
    ),
    axis=1,
)

# Calculate entry cost per position
positions["entry_cost"] = positions["total_cost"] / positions["quantity"]
positions["position_size"] = positions["market_value"]

print("📊 Breakeven Analysis")
display(
    positions[
        ["ticker", "option_type", "strike", "price_per_contract", "breakeven_price"]
    ].round(2)
)

# Current MSFT price vs breakeven
if "MSFT" in positions["ticker"].values:
    msft_current = yf.Ticker("MSFT").info.get("currentPrice", 0)
    print(f"\n💡 MSFT Current: ${msft_current:.2f}")
    msft_positions = positions[positions["ticker"] == "MSFT"]
    for _, pos in msft_positions.iterrows():
        distance = msft_current - pos["breakeven_price"]
        print(
            f"   {pos['strike']:.0f} Call: B/E ${pos['breakeven_price']:.2f}, Distance: ${distance:.2f}"
        )

📊 Breakeven Analysis


,ticker,option_type,strike,price_per_contract,breakeven_price
0,MSFT,CALL,260.0,151.30,411.30
1,MSFT,CALL,290.0,124.18,414.18



💡 MSFT Current: $404.37
   260 Call: B/E $411.30, Distance: $-6.93
   290 Call: B/E $414.18, Distance: $-9.81


## Technical Analysis & Live Price Data

Fetch current prices, analyst targets, and technical indicators from Yahoo Finance.


In [32]:
# Cell 5: Fetch Technical Data
def fetch_technical_analysis(ticker):
    """Fetch comprehensive technical data from yfinance"""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")

        if hist.empty:
            print(f"❌ No data for {ticker}")
            return {}

        # Moving averages
        current_price = hist["Close"].iloc[-1]
        ma_20 = hist["Close"].rolling(20).mean().iloc[-1]
        ma_50 = hist["Close"].rolling(50).mean().iloc[-1]
        ma_200 = hist["Close"].rolling(200).mean().iloc[-1]

        # Price levels
        high_52w = hist["High"].max()
        low_52w = hist["Low"].min()
        recent_high = hist["High"].tail(20).max()
        recent_low = hist["Low"].tail(20).min()

        # Volatility (30-day)
        returns = hist["Close"].pct_change().dropna()
        volatility = returns.tail(30).std() * np.sqrt(252)  # Annualized

        # ATR (14-day)
        high_low = hist["High"] - hist["Low"]
        high_close = (hist["High"] - hist["Close"].shift()).abs()
        low_close = (hist["Low"] - hist["Close"].shift()).abs()
        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        atr = true_range.rolling(14).mean().iloc[-1]

        # Analyst data
        info = stock.info
        analyst_target = info.get("targetMeanPrice", np.nan)
        target_high = info.get("targetHighPrice", np.nan)
        target_low = info.get("targetLowPrice", np.nan)
        rec_mean = info.get("recommendationMean", np.nan)
        analyst_count = info.get("numberOfAnalystOpinions", 0)

        return {
            "ticker": ticker,
            "current_price": current_price,
            "ma_20": ma_20,
            "ma_50": ma_50,
            "ma_200": ma_200,
            "high_52w": high_52w,
            "low_52w": low_52w,
            "recent_high": recent_high,
            "recent_low": recent_low,
            "volatility": volatility,
            "atr": atr,
            "analyst_target": analyst_target,
            "target_high": target_high,
            "target_low": target_low,
            "recommendation": rec_mean,
            "analyst_count": analyst_count,
            "hist_data": hist,
        }

    except Exception as e:
        print(f"❌ Error fetching {ticker}: {e}")
        return {}


# Fetch data for all unique tickers
unique_tickers = positions["ticker"].unique()
print(f"📡 Fetching technical data for {len(unique_tickers)} ticker(s)...\n")

technical_data = {}
for ticker in unique_tickers:
    print(f"   Fetching {ticker}...")
    tech = fetch_technical_analysis(ticker)
    if tech:
        technical_data[ticker] = tech
        print(
            f"     ✓ Price: ${tech['current_price']:.2f}, Target: ${tech['analyst_target']:.2f}"
        )
        print(f"     ✓ MA(20): ${tech['ma_20']:.2f}, MA(50): ${tech['ma_50']:.2f}")
        print(f"     ✓ ATR: ${tech['atr']:.2f}, Vol: {tech['volatility']:.1%}")
    else:
        print(f"     ❌ Failed to fetch {ticker}")

print(f"\n✅ Fetched data for {len(technical_data)} ticker(s)\n")

# Add current price to positions
positions["current_underlying"] = positions["ticker"].map(
    {k: v["current_price"] for k, v in technical_data.items()}
)
positions["analyst_target"] = positions["ticker"].map(
    {k: v.get("analyst_target", np.nan) for k, v in technical_data.items()}
)
positions["distance_to_breakeven"] = (
    positions["current_underlying"] - positions["breakeven_price"]
)

📡 Fetching technical data for 1 ticker(s)...

   Fetching MSFT...
     ✓ Price: $404.37, Target: $596.00
     ✓ MA(20): $438.14, MA(50): $464.41
     ✓ ATR: $16.43, Vol: 40.4%

✅ Fetched data for 1 ticker(s)



## Calculate Greeks

Option price sensitivity metrics (Delta, Gamma, Theta, Vega) using Black-Scholes model.


In [33]:
# Cell 6: Greeks Calculation
def calculate_greeks(row, risk_free_rate):
    """Calculate option Greeks using Black-Scholes"""
    try:
        if pd.isna(row["current_underlying"]) or pd.isna(row["dte"]) or row["dte"] <= 0:
            return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan])

        S = row["current_underlying"]
        K = row["strike"]
        T = row["dte"] / 365.0
        r = risk_free_rate

        # Get volatility from technical data
        tech = technical_data.get(row["ticker"], {})
        sigma = tech.get("volatility", 0.3)  # Default 30% if not available

        # Black-Scholes calculations
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)

        if row["option_type"] == "CALL":
            delta = norm.cdf(d1)
            theta = (
                -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                - r * K * np.exp(-r * T) * norm.cdf(d2)
            ) / 365
        else:
            delta = norm.cdf(d1) - 1
            theta = (
                -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                + r * K * np.exp(-r * T) * norm.cdf(-d2)
            ) / 365

        gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T) * 0.01  # 1% vol change

        return pd.Series([delta, gamma, theta, vega, sigma])

    except:
        return pd.Series([np.nan, np.nan, np.nan, np.nan, np.nan])


print("🧮 Calculating Option Greeks...\n")

# Calculate Greeks for all positions
positions[["delta", "gamma", "theta", "vega", "implied_vol"]] = positions.apply(
    lambda row: calculate_greeks(row, RISK_FREE_RATE), axis=1
)

# Position Greeks (per contract vs total)
positions["position_delta"] = positions["delta"] * positions["quantity"] * 100
positions["position_gamma"] = positions["gamma"] * positions["quantity"] * 100
positions["position_theta"] = positions["theta"] * positions["quantity"] * 100
positions["position_vega"] = positions["vega"] * positions["quantity"] * 100

print("✅ Greeks calculated\n")

# Greeks summary
greeks_display = positions[
    ["ticker", "option_type", "strike", "delta", "gamma", "theta", "vega"]
].round(4)
print(f"📊 Greeks Summary:")

display(greeks_display)

# Portfolio Greeks
print(f"\n📈 Portfolio Greeks (Total):")
portfolio_greeks = {
    "Delta": positions["position_delta"].sum(),
    "Gamma": positions["position_gamma"].sum(),
    "Theta": positions["position_theta"].sum(),
    "Vega": positions["position_vega"].sum(),
}
for greek, value in portfolio_greeks.items():
    print(f"   {greek}: {value:.2f}")

🧮 Calculating Option Greeks...

✅ Greeks calculated

📊 Greeks Summary:


,ticker,option_type,strike,delta,gamma,theta,vega
0,MSFT,CALL,260.0,0.9511,0.0008,-0.0577,0.3175
1,MSFT,CALL,290.0,0.9042,0.0013,-0.0783,0.5325



📈 Portfolio Greeks (Total):
   Delta: 185.53
   Gamma: 0.21
   Theta: -13.59
   Vega: 85.00


## 🎯 Trading Cheat Sheet

Quick reference for key price levels, targets, and P&L scenarios.


## Stop Loss Levels

Technical-based stop levels using recent lows and ATR.


In [34]:
# Cell 9: Stop Loss Calculator
def calculate_stop_levels(row):
    """Calculate conservative stop loss levels"""
    entry_price = row["price_per_contract"]
    breakeven = row["breakeven_price"]

    # Conservative stops (adjust based on your risk tolerance)
    # Initial stop: 15% below entry
    # Breakeven stop: Move to breakeven + small buffer when profitable
    initial_stop = entry_price * 0.85
    breakeven_stop = breakeven + (breakeven * 0.02)  # 2% buffer

    return pd.Series([initial_stop, breakeven_stop])


positions[["stop_initial", "stop_breakeven"]] = positions.apply(
    calculate_stop_levels, axis=1
)

# Display stop levels
stop_display = positions[
    [
        "ticker",
        "strike",
        "price_per_contract",
        "breakeven_price",
        "stop_initial",
        "stop_breakeven",
    ]
].round(2)
print("🛑 Recommended Stop Levels:")
print("   • Initial Stop: 85% of entry price (15% max loss)")
print("   • Breakeven Stop: Move to breakeven + 2% when profitable")
print()
display(stop_display)

🛑 Recommended Stop Levels:
   • Initial Stop: 85% of entry price (15% max loss)
   • Breakeven Stop: Move to breakeven + 2% when profitable



,ticker,strike,price_per_contract,breakeven_price,stop_initial,stop_breakeven
0,MSFT,260.0,151.30,411.30,128.61,419.53
1,MSFT,290.0,124.18,414.18,105.55,422.46


## DTE and Time Decay Monitoring

Days to expiration and time decay risk visualization.


In [35]:
# Cell 10: DTE Analysis
print("⏳ Days to Expiration Analysis:\n")

# DTE summary
dte_summary = positions[
    ["ticker", "symbol", "strike", "dte", "market_value", "theta"]
].copy()
dte_summary["theta_per_day"] = dte_summary["theta"] * dte_summary["market_value"]

# Sort by DTE
dte_summary = dte_summary.sort_values("dte")

print("📅 Positions by Days to Expiration:")
display(dte_summary.round(4))

# Time decay chart
print("\n📉 Time Decay (Theta) Impact:")
theta_display = positions[["ticker", "strike", "dte", "theta", "position_theta"]].copy()
theta_display["daily_decay_$"] = theta_display["position_theta"]
display(theta_display.round(4))

# Warning for short DTE
short_dte = positions[positions["dte"] <= 30]
if not short_dte.empty:
    print(f"\n⚠️  WARNING: {len(short_dte)} position(s) with 30 DTE or less")
    print("   Time decay accelerates significantly in the last 30 days!")
    display(short_dte[["ticker", "strike", "dte", "theta"]].round(4))

⏳ Days to Expiration Analysis:

📅 Positions by Days to Expiration:


,ticker,symbol,strike,dte,market_value,theta,theta_per_day
0,MSFT,MSFT 09/18/2026 260.00 C,260.0,219,15130.0,-0.0577,-872.3113
1,MSFT,MSFT 09/18/2026 290.00 C,290.0,219,12417.5,-0.0783,-972.1770



📉 Time Decay (Theta) Impact:


,ticker,strike,dte,theta,position_theta,daily_decay_$
0,MSFT,260.0,219,-0.0577,-5.7654,-5.7654
1,MSFT,290.0,219,-0.0783,-7.8291,-7.8291


## 📊 Portfolio Summary & Next Actions

Key takeaways and recommended actions.


In [36]:
# Cell 11: Portfolio Summary
total_cost = positions["total_cost"].sum()
total_value = positions["market_value"].sum()
total_pl = positions["unrealized_pl"].sum()
total_pl_pct = (total_pl / total_cost) * 100

print("💼 Portfolio Summary")
print(f"   Total Cost Basis: ${total_cost:,.2f}")
print(f"   Current Value: ${total_value:,.2f}")
print(f"   Total P&L: ${total_pl:,.2f} ({total_pl_pct:.2f}%)")
print()

# Group by ticker
print("📈 Exposure by Ticker:")
ticker_exposure = (
    positions.groupby("ticker")
    .agg({"market_value": "sum", "unrealized_pl": "sum", "position_delta": "sum"})
    .round(2)
)
display(ticker_exposure)

# Export positions
output_file = f"outputs/options_monitor_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
positions.to_csv(output_file, index=False)
print(f"\n💾 Full analysis exported to: {output_file}")

print("\n✅ Analysis Complete!")
print(f"📅 Refresh this notebook daily for updated prices.")

💼 Portfolio Summary
   Total Cost Basis: $27,475.32
   Current Value: $27,547.50
   Total P&L: $72.18 (0.26%)

📈 Exposure by Ticker:


,market_value,unrealized_pl,position_delta
ticker,,,
MSFT,27547.5,72.18,185.53



💾 Full analysis exported to: outputs/options_monitor_20260211_230318.csv

✅ Analysis Complete!
📅 Refresh this notebook daily for updated prices.


## 🎲 Probability Analysis & Statistical Edge

Risk-neutral probability analysis using Black-Scholes framework and Monte Carlo simulation.


In [37]:
# Cell 12: Probability Analysis
def risk_neutral_probability(S, K, T, r, vol):
    # Risk-neutral probability that S_T > K at expiration
    if T <= 0 or vol <= 0:
        return np.nan
    d2 = (np.log(S / K) + (r - 0.5 * (vol**2)) * T) / (vol * np.sqrt(T))
    return norm.cdf(-d2)


def calculate_price_target_probabilities(row, tech):
    # Price levels to analyze
    S = row["current_underlying"]
    T = row["dte"] / 365.0
    r = RISK_FREE_RATE
    vol = tech.get("volatility", 0.3)

    targets = {
        "breakeven": row["breakeven_price"],
        "target_450": 450.0,
        "target_500": 500.0,
        "target_550": 550.0,
        "analyst_target": tech.get("analyst_target", 0),
        "target_650": 650.0,
        "target_700": 700.0,
        "analyst_high": tech.get("target_high", 0),
    }

    prob_dict = {}
    for name, level in targets.items():
        if level > 0:
            prob = risk_neutral_probability(S, level, T, r, vol)
            prob_dict[f"prob_{name}"] = prob

    return pd.Series(prob_dict)


def calculate_expected_value(row, tech):
    # Calculate expected value across analyst scenarios
    entry_cost = row["price_per_contract"]
    strike = row["strike"]
    current = tech.get("current_price", 0)

    # Scenarios based on analyst range
    scenarios = {
        "bear_case": {
            "price": tech.get("target_low", current * 0.85),
            "prob": 0.25,
        },
        "base_case": {
            "price": tech.get("analyst_target", current * 1.10),
            "prob": 0.50,
        },
        "bull_case": {
            "price": tech.get("target_high", current * 1.25),
            "prob": 0.25,
        },
    }

    expected_value = 0
    for scenario_name, scenario in scenarios.items():
        target_price = scenario["price"]
        probability = scenario["prob"]

        if target_price > 0 and row["option_type"] == "CALL":
            payoff = max(0, target_price - strike) - entry_cost
            expected_value += payoff * probability

    statistical_edge = (expected_value / entry_cost) * 100 if entry_cost > 0 else np.nan

    return pd.Series(
        {
            "expected_value": expected_value,
            "statistical_edge_pct": statistical_edge,
        }
    )


print("Calculating probabilities...")
prob_results = []
for _, row in positions.iterrows():
    tech = technical_data.get(row["ticker"], {})
    if tech:
        probs = calculate_price_target_probabilities(row, tech)
        prob_results.append(probs)
    else:
        prob_results.append(pd.Series())

prob_df = pd.DataFrame(prob_results)
positions = pd.concat([positions, prob_df], axis=1)

print("Calculating expected values...")
ev_results = []
for _, row in positions.iterrows():
    tech = technical_data.get(row["ticker"], {})
    if tech:
        ev = calculate_expected_value(row, tech)
        ev_results.append(ev)
    else:
        ev_results.append(pd.Series())

ev_df = pd.DataFrame(ev_results)
positions = pd.concat([positions, ev_df], axis=1)

# Show results
prob_cols = [c for c in positions.columns if c.startswith("prob_")]
print("Probability Summary:")
display(positions[["ticker", "strike"] + prob_cols].round(3))

print("Statistical Edge:")
ev_cols = ["expected_value", "statistical_edge_pct"]
display(positions[["ticker", "strike"] + ev_cols].round(2))


Calculating probabilities...
Calculating expected values...
Probability Summary:


,ticker,strike,prob_breakeven,prob_target_450,prob_target_500,prob_target_550,prob_analyst_target,prob_target_650,prob_target_700,prob_analyst_high
0,MSFT,260.0,0.549,0.66,0.773,0.854,0.905,0.944,0.966,0.975
1,MSFT,290.0,0.558,0.66,0.773,0.854,0.905,0.944,0.966,0.975


Statistical Edge:


,ticker,strike,expected_value,statistical_edge_pct
0,MSFT,260.0,167.20,110.51
1,MSFT,290.0,164.32,132.33


## 🔮 Monte Carlo Simulation

10,000 price path simulations to validate analytical probabilities.


In [38]:
# Cell 13: Monte Carlo
def run_monte_carlo(ticker, tech, n_sim=10000):
    if not tech:
        return None

    S0 = tech["current_price"]
    vol = tech.get("volatility", 0.3)
    r = RISK_FREE_RATE

    ticker_pos = positions[positions["ticker"] == ticker]
    if ticker_pos.empty:
        return None

    max_dte = ticker_pos["dte"].max()
    n_days = max_dte

    if n_days <= 0:
        return None

    T = n_days / 365.0
    dt = 1 / 365.0
    n_steps = int(n_days)

    np.random.seed(42)
    shocks = np.random.normal(0, 1, (n_sim, n_steps))

    price_paths = np.zeros((n_sim, n_steps))
    price_paths[:, 0] = S0

    time_array = np.linspace(0, T, n_steps)

    for i in range(1, n_steps):
        dt_step = time_array[i] - time_array[i - 1]
        price_paths[:, i] = price_paths[:, i - 1] * np.exp(
            (r - 0.5 * vol**2) * dt_step + vol * np.sqrt(dt_step) * shocks[:, i - 1]
        )

    return {
        "price_paths": price_paths,
        "terminal_prices": price_paths[:, -1],
        "n_simulations": n_sim,
        "time_array": time_array,
        "volatility": vol,
    }


def calculate_mc_probabilities(sim_result, levels):
    if not sim_result:
        return {}

    terminals = sim_result["terminal_prices"]
    result = {}

    for name, level in levels.items():
        if level > 0:
            result[f"mc_prob_{name}"] = (terminals >= level).mean()

    return result


# Run Monte Carlo
print("Running Monte Carlo simulations...")
mc_results = {}

for ticker in unique_tickers:
    tech = technical_data.get(ticker)
    if not tech:
        continue

    ticker_pos = positions[positions["ticker"] == ticker]
    if ticker_pos.empty:
        continue

    print(f"  Simulating {ticker}...")
    result = run_monte_carlo(ticker, tech)

    if result:
        mc_results[ticker] = result

        # Calculate probabilities
        price_levels = {
            "breakeven": ticker_pos["breakeven_price"].iloc[0],
            "analyst_target": tech.get("analyst_target", 0),
        }

        mc_probs = calculate_mc_probabilities(result, price_levels)

        # Store
        for pos_idx in ticker_pos.index:
            for prob_name, prob_val in mc_probs.items():
                positions.loc[pos_idx, prob_name] = prob_val

        print(f"    Terminal mean: ${result['terminal_prices'].mean():.2f}")
        print(f"    Terminal std: ${result['terminal_prices'].std():.2f}")

print("\nComparing Analytical vs Monte Carlo:")
comp_cols = [
    c for c in positions.columns if c.startswith("prob_") or c.startswith("mc_prob_")
]
display(positions[["ticker", "strike"] + comp_cols].round(3))


Running Monte Carlo simulations...
  Simulating MSFT...
    Terminal mean: $413.09
    Terminal std: $129.53

Comparing Analytical vs Monte Carlo:


,ticker,strike,prob_breakeven,prob_target_450,prob_target_500,prob_target_550,prob_analyst_target,prob_target_650,prob_target_700,prob_analyst_high,mc_prob_breakeven,mc_prob_analyst_target
0,MSFT,260.0,0.549,0.66,0.773,0.854,0.905,0.944,0.966,0.975,0.446,0.09
1,MSFT,290.0,0.558,0.66,0.773,0.854,0.905,0.944,0.966,0.975,0.446,0.09


## ⏱️ Probability Time Evolution

How probabilities decay over time (time decay impact on odds).


In [39]:
# Cell 14: Time Evolution
def calc_time_evolution(row, tech):
    if not tech or row["dte"] <= 0:
        return pd.DataFrame()

    S = row["current_underlying"]
    r = RISK_FREE_RATE
    vol = tech.get("volatility", 0.3)

    targets = [
        (row["breakeven_price"], "breakeven"),
        (500.0, "target_500"),
        (tech.get("analyst_target", 0), "analyst_target")
        if tech.get("analyst_target", 0) > 0
        else None,
    ]
    targets = [t for t in targets if t and t[0] > 0]

    time_steps = [180, 150, 120, 90, 60, 30, 14, 7]
    time_steps = [t for t in time_steps if t <= row["dte"]]
    time_steps.append(row["dte"])
    time_steps.sort()

    results = []
    for days in time_steps:
        T = days / 365.0
        data = {"dte": days}

        for level, name in targets:
            # Use risk neutral probability function
            d2 = (np.log(S / level) + (r - 0.5 * (vol**2)) * T) / (vol * np.sqrt(T))
            prob = norm.cdf(-d2)
            data[f"prob_{name}"] = prob

        results.append(data)

    return pd.DataFrame(results)


print("Calculating probability evolution...")
evo_results = {}

for _, row in positions.iterrows():
    tech = technical_data.get(row["ticker"])
    if not tech:
        continue

    print(f"  {row['ticker']} {row['strike']:.0f} call...")
    evo_df = calc_time_evolution(row, tech)

    if not evo_df.empty:
        key = f"{row['ticker']}_{row['strike']}"
        evo_results[key] = evo_df

        print(
            f"    Current breakeven prob: {evo_df.iloc[-1].get('prob_breakeven', 0):.1%}"
        )

print("\nCurrent Probabilities:")
current_probs = []
for key, df in evo_results.items():
    ticker, strike = key.split("_")
    current_row = df.iloc[-1]
    current_probs.append(
        {
            "Ticker": ticker,
            "Strike": strike,
            "Breakeven": current_row.get("prob_breakeven"),
            "Target_500": current_row.get("prob_target_500"),
            "Analyst_Target": current_row.get("prob_analyst_target"),
        }
    )

current_df = pd.DataFrame(current_probs)
display(current_df.round(3))


Calculating probability evolution...
  MSFT 260 call...
    Current breakeven prob: 54.9%
  MSFT 290 call...
    Current breakeven prob: 55.8%

Current Probabilities:


,Ticker,Strike,Breakeven,Target_500,Analyst_Target
0,MSFT,260.0,0.549,0.773,0.905
1,MSFT,290.0,0.558,0.773,0.905


## 📈 Enhanced Summary with Statistical Edge

Probability-weighted analysis and key insights.


In [40]:
# Cell 15: Enhanced Summary
print("Enhanced Portfolio Summary:")
print()

total_cost = positions["total_cost"].sum()
total_value = positions["market_value"].sum()
total_pl = positions["unrealized_pl"].sum()
total_ev = positions["expected_value"].sum()

print(f"Total Cost Basis: ${total_cost:,.2f}")
print(f"Current Value: ${total_value:,.2f}")
print(f"Current P&L: ${total_pl:,.2f}")
print(f"Expected Value: ${total_ev:,.2f}")
print()

print("Per Position Analysis:")
cols = [
    "ticker",
    "strike",
    "price_per_contract",
    "current_underlying",
    "statistical_edge_pct",
    "prob_breakeven",
    "prob_analyst_target",
]
a = positions[cols].copy()
a["statistical_edge_pct"] = a["statistical_edge_pct"].round(1)
a["prob_breakeven"] = a["prob_breakeven"].round(1)
a["prob_analyst_target"] = a["prob_analyst_target"].round(1)
display(a)

# Save enhanced data
output_file = f"outputs/options_monitor_enhanced_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
positions.to_csv(output_file, index=False)
print(f"\nEnhanced data saved to: {output_file}")
print("\n✅ Probability Analysis Complete!")


Enhanced Portfolio Summary:

Total Cost Basis: $27,475.32
Current Value: $27,547.50
Current P&L: $72.18
Expected Value: $331.52

Per Position Analysis:


,ticker,strike,price_per_contract,current_underlying,statistical_edge_pct,prob_breakeven,prob_analyst_target
0,MSFT,260.0,151.300,404.369995,110.5,0.5,0.9
1,MSFT,290.0,124.175,404.369995,132.3,0.6,0.9



Enhanced data saved to: outputs/options_monitor_enhanced_20260211_230319.csv

✅ Probability Analysis Complete!


## 📊 Probability Visualization Suite

Interactive charts showing probability windows, confidence bands, and price evolution over time.

In [42]:
# Cell 16: Probability Waterfall Chart
def create_probability_waterfall_chart(ticker, positions_df, technical_data):
    tech = technical_data.get(ticker)
    if not tech:
        return None
    
    ticker_positions = positions_df[positions_df['ticker'] == ticker]
    if ticker_positions.empty:
        return None
    
    max_dte = ticker_positions['dte'].max()
    if max_dte <= 0:
        return None
    
    time_checkpoints = [0, 30, 60, 90, 120, 150, 180, max_dte]
    time_checkpoints = sorted(list(set([t for t in time_checkpoints if t <= max_dte])))
    
    S = tech['current_price']
    vol = tech.get('volatility', 0.3)
    r = RISK_FREE_RATE
    
    targets = {}
    for _, pos in ticker_positions.iterrows():
        targets[f"Breakeven {pos['strike']:.0f}C"] = pos['breakeven_price']
    
    targets.update({
        'Target $500': 500.0,
        'Target $550': 550.0,
        'Analyst $596': tech.get('analyst_target', 0),
    })
    
    targets = {k: v for k, v in targets.items() if v > 0}
    prob_data = []
    
    for days in time_checkpoints:
        T = days / 365.0 if days > 0 else 0.001
        row = {'days_from_now': days, 'dte': max_dte - days}
        for target_name, target_price in targets.items():
            if T > 0:
                d2 = (np.log(S / target_price) + (r - 0.5 * vol**2) * T) / (vol * np.sqrt(T))
                prob = norm.cdf(-d2)
            else:
                prob = 1.0 if S >= target_price else 0.0
            row[f'prob_{target_name}'] = prob
        prob_data.append(row)
    
    prob_df = pd.DataFrame(prob_data)
    fig = go.Figure()
    
    colors = {
        'Breakeven': COLORS['breakeven'],
        'Target $500': 'orange',
        'Target $550': 'green',
        'Analyst $596': COLORS['target'],
    }
    
    for col in prob_df.columns:
        if col.startswith('prob_'):
            target_name = col.replace('prob_', '')
            color = colors.get(target_name, 'gray')
            fig.add_trace(go.Scatter(
                x=prob_df['days_from_now'],
                y=prob_df[col] * 100,
                mode='lines+markers',
                name=target_name,
                line=dict(color=color, width=3),
                marker=dict(size=8),
                hovertemplate=f'<b>{target_name}</b><br>Days: %{{x}}<br>Probability: %{{y:.1f}}%<extra></extra>'
            ))
    
    fig.add_vline(x=0, line=dict(color='red', width=2, dash='dash'), annotation_text=f"Today (DTE: {max_dte})", annotation_position="top")
    fig.update_layout(title=f'{ticker} Probability Evolution Waterfall', xaxis_title='Days from Now', yaxis_title='Probability of Hitting Target (%)', template='plotly_dark', height=500, hovermode='x unified')
    
    return fig, prob_df

print("📊 Creating Probability Waterfall Charts...
")
for ticker in unique_tickers:
    print(f"Charting {ticker}...")
    fig, prob_df = create_probability_waterfall_chart(ticker, positions, technical_data)
    if fig:
        fig.show()
        print(f"\n{ticker} Probability Data:")
        display_cols = ['days_from_now', 'dte'] + [c for c in prob_df.columns if c.startswith('prob_')]
        display_df = prob_df[display_cols].copy()
        for col in display_df.columns:
            if col.startswith('prob_'):
                display_df[col] = (display_df[col] * 100).round(1)
        display(display_df)
        print(f"✅ {ticker} waterfall chart created\n")
    else:
        print(f"❌ Failed to create {ticker} chart\n")


SyntaxError: unterminated string literal (detected at line 138) (1320411285.py, line 138)

## 📈 Confidence Band Evolution

Visual probability windows showing price uncertainty ranges over time with your position levels.

In [ ]:
# Cell 17: Confidence Band Evolution Chart
def create_confidence_band_chart(ticker, positions_df, technical_data):
    tech = technical_data.get(ticker)
    if not tech:
        return None
    
    ticker_positions = positions_df[positions_df['ticker'] == ticker]
    if ticker_positions.empty:
        return None
    
    max_dte = ticker_positions['dte'].max()
    if max_dte <= 0:
        return None
    
    S0 = tech['current_price']
    vol = tech.get('volatility', 0.3)
    r = RISK_FREE_RATE
    
    time_days = np.arange(0, max_dte + 1, 7)
    time_years = time_days / 365.0
    
    expected_path = S0 * np.exp(r * time_years)
    std_devs = S0 * np.sqrt(np.exp(vol**2 * time_years) - 1) * np.exp(r * time_years)
    
    ci_68_low = np.maximum(expected_path - std_devs, 0)
    ci_68_high = expected_path + std_devs
    ci_95_low = np.maximum(expected_path - 2 * std_devs, 0)
    ci_95_high = expected_path + 2 * std_devs
    ci_99_low = np.maximum(expected_path - 3 * std_devs, 0)
    ci_99_high = expected_path + 3 * std_devs
    
    fig = go.Figure()
    
    # Bands
    fig.add_trace(go.Scatter(x=time_days, y=ci_99_high, fill=None, mode='lines', line=dict(color='rgba(0,0,0,0)'), showlegend=False, hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=time_days, y=ci_99_low, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'), fillcolor='rgba(100,100,255,0.1)', name='99% CI', hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=time_days, y=ci_95_high, fill=None, mode='lines', line=dict(color='rgba(0,0,0,0)'), showlegend=False, hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=time_days, y=ci_95_low, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'), fillcolor='rgba(100,200,255,0.15)', name='95% CI', hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=time_days, y=ci_68_high, fill=None, mode='lines', line=dict(color='rgba(0,0,0,0)'), showlegend=False, hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=time_days, y=ci_68_low, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'), fillcolor='rgba(100,255,255,0.2)', name='68% CI', hoverinfo='skip'))
    
    # Expected path
    fig.add_trace(go.Scatter(x=time_days, y=expected_path, mode='lines', line=dict(color='white', width=2, dash='dash'), name='Expected Path', hovertemplate='Day %{x}<br>Expected: $%{y:.2f}<extra></extra>'))
    
    # Current price
    fig.add_hline(y=S0, line=dict(color='green', width=2), annotation_text=f'Current: ${S0:.2f}', annotation_position='left')
    
    # Position levels
    for _, pos in ticker_positions.iterrows():
        fig.add_hline(y=pos['breakeven_price'], line=dict(color=COLORS['breakeven'], width=2, dash='dash'), annotation_text=f"B/E ${pos['breakeven_price']:.0f}", annotation_position='right')
        fig.add_hline(y=pos['strike'], line=dict(color='rgba(255,255,0,0.5)', width=1, dash='dot'), annotation_text=f"Strike ${pos['strike']:.0f}", annotation_position='right')
    
    if not pd.isna(tech.get('analyst_target')):
        fig.add_hline(y=tech['analyst_target'], line=dict(color=COLORS['target'], width=3), annotation_text=f"Analyst ${tech['analyst_target']:.0f}", annotation_position='right')
    
    fig.update_layout(title=f'{ticker} Price Confidence Bands', xaxis_title='Days from Now', yaxis_title='Stock Price ($)', template='plotly_dark', height=600, hovermode='x unified')
    
    return fig

print("📈 Creating Confidence Band Evolution Charts...
")
for ticker in unique_tickers:
    print(f"Charting {ticker}...")
    fig = create_confidence_band_chart(ticker, positions, technical_data)
    if fig:
        fig.show()
        print(f"✅ {ticker} confidence bands created
")
    else:
        print(f"❌ Failed to create {ticker} chart
")


## ⚡ Realized vs Expected Path Tracker

Compare actual stock performance against model predictions with daily updating confidence bands.

In [ ]:
# Cell 18: Realized vs Expected Path Tracker
def create_realized_tracker(ticker, positions_df, technical_data, days_history=60):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=f"{days_history}d")
    
    if hist.empty:
        print(f"  ⚠️  No historical data for {ticker}, skipping tracker")
        return None
    
    hist = hist.reset_index()
    ticker_positions = positions_df[positions_df['ticker'] == ticker]
    first_pos = ticker_positions.iloc[0]
    
    tech = technical_data.get(ticker, {})
    if not tech:
        return None
    
    S0 = tech.get('current_price', hist['Close'].iloc[-1])
    vol = tech.get('volatility', 0.3)
    r = RISK_FREE_RATE
    
    hist['days_from_entry'] = (hist['Date'] - hist['Date'].iloc[0]).dt.days
    hist['years_from_entry'] = hist['days_from_entry'] / 365.0
    hist['expected_price'] = S0 * np.exp(r * hist['years_from_entry'])
    
    hist['price_std'] = S0 * np.sqrt(np.exp(vol**2 * hist['years_from_entry']) - 1) * np.exp(r * hist['years_from_entry'])
    hist['ci_95_low'] = hist['expected_price'] - 2 * hist['price_std']
    hist['ci_95_high'] = hist['expected_price'] + 2 * hist['price_std']
    hist['ci_68_low'] = hist['expected_price'] - hist['price_std']
    hist['ci_68_high'] = hist['expected_price'] + hist['price_std']
    
    hist['deviation'] = ((hist['Close'] - hist['expected_price']) / hist['expected_price']) * 100
    hist['vs_breakeven'] = hist['Close'] - first_pos['breakeven_price']
    
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.08, subplot_titles=(f'{ticker} Realized vs Expected Path', 'Deviation from Expected (%)', 'vs Breakeven Target'), row_heights=[0.5, 0.25, 0.25])
    
    # Confidence bands
    fig.add_trace(go.Scatter(x=hist['Date'], y=hist['ci_95_high'], fill=None, mode='lines', line=dict(color='rgba(100,150,255,0.3)', width=0), showlegend=True, name='95% CI', hovertemplate='95% CI High: $%{y:.2f}<extra></extra>'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['Date'], y=hist['ci_95_low'], fill='tonexty', mode='lines', line=dict(color='rgba(100,150,255,0.3)', width=0), fillcolor='rgba(100,150,255,0.1)', showlegend=False, name='95% CI Low', hovertemplate='95% CI Low: $%{y:.2f}<extra></extra>'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['Date'], y=hist['ci_68_high'], fill=None, mode='lines', line=dict(color='rgba(100,200,255,0.5)', width=0), showlegend=True, name='68% CI', hovertemplate='68% CI High: $%{y:.2f}<extra></extra>'), row=1, col=1)
    fig.add_trace(go.Scatter(x=hist['Date'], y=hist['ci_68_low'], fill='tonexty', mode='lines', line=dict(color='rgba(100,200,255,0.5)', width=0), fillcolor='rgba(100,200,255,0.15)', showlegend=False, name='68% CI Low', hovertemplate='68% CI Low: $%{y:.2f}<extra></extra>'), row=1, col=1)
    
    # Expected path
    fig.add_trace(go.Scatter(x=hist['Date'], y=hist['expected_price'], mode='lines', line=dict(color='gray', width=2, dash='dash'), name='Expected Path', hovertemplate='Expected: $%{y:.2f}<extra></extra>'), row=1, col=1)
    
    # Realized path
    hist['color'] = hist['deviation'].apply(lambda x: 'green' if x >= 0 else 'red')
    for color in ['green', 'red']:
        color_mask = hist['color'] == color
        if color_mask.any():
            fig.add_trace(go.Scatter(x=hist.loc[color_mask, 'Date'], y=hist.loc[color_mask, 'Close'], mode='lines+markers', line=dict(color=color, width=3), marker=dict(size=4, color=color), name=f'Realized Price ({color})', hovertemplate='Date: %{x}<br>Price: $%{y:.2f}<br>Deviation: %{customdata:.1f}%<extra></extra>', customdata=hist.loc[color_mask, 'deviation']), row=1, col=1)
    
    # Breakeven level
    be_prices = ticker_positions['breakeven_price'].unique()
    for be_price in be_prices:
        fig.add_hline(y=be_price, line=dict(color=COLORS['breakeven'], width=2, dash='dashdot'), annotation_text=f"Breakeven ${be_price:.0f}", row=1, col=1)
    
    # Deviation chart
    fig.add_trace(go.Scatter(x=hist['Date'], y=hist['deviation'], mode='lines+markers', line=dict(color='lightblue', width=2), marker=dict(size=4), name='Deviation %', hovertemplate='Date: %{x}<br>Deviation: %{y:.1f}%<extra></extra>'), row=2, col=1)
    fig.add_hline(y=0, line=dict(color='gray', width=1), row=2, col=1)
    
    # vs Breakeven chart
    colors_be = hist['vs_breakeven'].apply(lambda x: 'green' if x >= 0 else 'red')
    for color in ['green', 'red']:
        color_mask = colors_be == color
        if color_mask.any():
            fig.add_trace(go.Scatter(x=hist.loc[color_mask, 'Date'], y=hist.loc[color_mask, 'vs_breakeven'], mode='lines+markers', line=dict(color=color, width=2), marker=dict(size=4, color=color), name=f'vs Breakeven ({color})', hovertemplate='Date: %{x}<br>vs Breakeven: $%{y:.2f}<extra></extra>'), row=3, col=1)
    fig.add_hline(y=0, line=dict(color='yellow', width=1), row=3, col=1)
    
    # Add annotation
    latest_price = hist['Close'].iloc[-1]
    latest_deviation = hist['deviation'].iloc[-1]
    days_tracking = hist['days_from_entry'].iloc[-1]
    
    fig.add_annotation(x=hist['Date'].iloc[-1], y=latest_price, text=f"Latest: ${latest_price:.2f}<br>({latest_deviation:+.1f}% from exp)", showarrow=True, arrowhead=2, bgcolor="rgba(255,255,255,0.1)", bordercolor="white", row=1, col=1)
    
    fig.update_layout(title=f'{ticker} Realized vs Expected Path ({days_tracking:.0f} days tracked)', template='plotly_dark', height=800, hovermode='x unified')
    fig.update_xaxes(title_text="Date", row=3, col=1)
    fig.update_yaxes(title_text="Stock Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Deviation (%)", row=2, col=1)
    fig.update_yaxes(title_text="vs Breakeven ($)", row=3, col=1)
    
    return fig, hist

print("⚡ Creating Realized vs Expected Trackers...
")
for ticker in unique_tickers:
    print(f"Tracking {ticker}...")
    fig, hist_df = create_realized_tracker(ticker, positions, technical_data)
    if fig:
        fig.show()
        print(f"\n{ticker} Tracking Summary:")
        print(f"  Latest price: ${hist_df['Close'].iloc[-1]:.2f}")
        print(f"  vs Expected: {hist_df['deviation'].iloc[-1]:+.1f}%")
        print(f"  Tracking period: {hist_df['days_from_entry'].iloc[-1]:.0f} days")
        print(f"✅ {ticker} tracker created\n")
    else:
        print(f"❌ Failed to create {ticker} tracker\n")


## 🎮 Dynamic Probability Explorer

Interactive slider to explore how probabilities change with different DTE scenarios.

In [ ]:
# Cell 19: Interactive Probability Slider
def create_probability_slider(ticker, positions_df, technical_data):
    tech = technical_data.get(ticker)
    if not tech:
        return None
    
    ticker_positions = positions_df[positions_df['ticker'] == ticker]
    if ticker_positions.empty:
        return None
    
    max_dte = ticker_positions['dte'].max()
    S = tech['current_price']
    vol = tech.get('volatility', 0.3)
    r = RISK_FREE_RATE
    
    price_min = int(S * 0.6)
    price_max = int(S * 1.8)
    price_range = np.arange(price_min, price_max, 2)
    
    dte_scenarios = [7, 14, 30, 60, 90, 120, 150, 180, max_dte]
    dte_scenarios = sorted(list(set([t for t in dte_scenarios if t <= max_dte])))
    
    distributions = {}
    for dte in dte_scenarios:
        T = dte / 365.0
        mu = np.log(S) + (r - 0.5 * vol**2) * T
        sigma = vol * np.sqrt(T)
        pdf = np.exp(-(np.log(price_range) - mu)**2 / (2 * sigma**2)) / (price_range * sigma * np.sqrt(2 * np.pi))
        distributions[dte] = {'prices': price_range, 'pdf': pdf, 'pdf_max': pdf.max()}
    
    fig = make_subplots(rows=2, cols=1, subplot_titles=('Probability Distribution', 'Cumulative Probabilities'), row_heights=[0.7, 0.3], vertical_spacing=0.1)
    
    init_dte = max_dte
    init_dist = distributions[init_dte]
    T_init = init_dte / 365.0
    
    fig.add_trace(go.Scatter(x=init_dist['prices'], y=init_dist['pdf'], mode='lines', fill='tozeroy', line=dict(color='lightblue', width=2), fillcolor='rgba(100,150,255,0.3)', name='Probability Density', hovertemplate='Price: $%{x:.0f}<br>Density: %{y:.4f}<extra></extra>'), row=1, col=1)
    
    be_price = ticker_positions['breakeven_price'].iloc[0]
    fig.add_vline(x=be_price, line=dict(color=COLORS['breakeven'], width=2, dash='dash'), annotation_text=f"Breakeven ${be_price:.0f}", row=1, col=1)
    
    if not pd.isna(tech.get('analyst_target')):
        target_price = tech['analyst_target']
        fig.add_vline(x=target_price, line=dict(color=COLORS['target'], width=2), annotation_text=f"Target ${target_price:.0f}", annotation_position="top", row=1, col=1)
        d2_target = (np.log(S / target_price) + (r - 0.5 * vol**2) * T_init) / (vol * np.sqrt(T_init))
        prob_target = norm.cdf(-d2_target) * 100
        fig.add_annotation(x=target_price * 1.1, y=init_dist['pdf_max'] * 0.8, text=f"P(≥${target_price:.0f}): {prob_target:.1f}%", showarrow=False, bgcolor="rgba(255,255,255,0.1)", row=1, col=1)
    
    # Cumulative probabilities
    key_levels = [
        (be_price, 'Breakeven', COLORS['breakeven']),
        (500.0, '$500', 'orange'),
        (tech.get('analyst_target', 0), 'Target', COLORS['target']) if not pd.isna(tech.get('analyst_target')) else None,
    ]
    key_levels = [l for l in key_levels if l is not None and l[0] > 0]
    
    prob_bars = []
    bar_colors = []
    bar_labels = []
    
    for level, label, color in key_levels:
        d2_level = (np.log(S / level) + (r - 0.5 * vol**2) * T_init) / (vol * np.sqrt(T_init))
        prob_level = norm.cdf(-d2_level) * 100
        prob_bars.append(prob_level)
        bar_colors.append(color)
        bar_labels.append(f"{label}<br>{prob_level:.1f}%")
    
    fig.add_trace(go.Bar(x=[l[1] for l in key_levels], y=prob_bars, marker_color=bar_colors, text=bar_labels, textposition='auto', name='Probability', hovertemplate='%{x}: %{y:.1f}%<extra></extra>'), row=2, col=1)
    
    fig.update_layout(title=f'{ticker} Probability Distribution (DTE: {init_dte} days)', template='plotly_dark', height=600, showlegend=True)
    fig.update_xaxes(title_text="Stock Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Probability Density", row=1, col=1)
    fig.update_xaxes(title_text="Price Level", row=2, col=1)
    fig.update_yaxes(title_text="Probability (%)", row=2, col=1)
    
    return fig

print("🎮 Creating Dynamic Probability Explorer...
")
for ticker in unique_tickers:
    print(f"Creating slider for {ticker}...")
    fig = create_probability_slider(ticker, positions, technical_data)
    if fig:
        fig.show()
        print(f"✅ {ticker} dynamic explorer created
")
    else:
        print(f"❌ Failed to create {ticker} explorer
")
